# FarmVibes.AI - SpaceEye & NDVI Timelapse

This notebook provides a self-contained workspace for generating and visualizing a timelapse of SpaceEye and NDVI data for a given time range and geometry. 

### Conda environment setup
To install the required packages, see [this README file](../README.md).

------------------------


### Imports & Constants

In [ ]:
from datetime import datetime

import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
import shapely as shpg
import yaml
from IPython.core.display import Image

from vibe_core.client import get_default_vibe_client
from vibe_notebook.plot import lw_plot
from vibe_notebook.raster import read_clip_index, read_raster, spaceeye_to_img

### Utility method

In [ ]:
def get_missing_ratio(raster, geometry):
    ar = read_raster(raster, geometry, filled=False)[0]
    c = ar.count()
    total = np.prod(ar.shape)
    return 1.0 - (c / total)

----------------

### Generate data with FarmVibes.AI platform

Let's define the region and the time range to consider for this task:
- **Region:** in this example, we define the geometry by the latitude and longitude coordinates of a bounding box;
- **Time Range:** we define the range as a tuple with two datetimes (start and end dates);

In [ ]:
bbox = (74.39184745, 18.16562525, 74.54009755, 18.26854175)
geometry = shpg.geometry.box(*bbox, ccw=True)

time_range = (datetime(2021, 1, 1), datetime(2022, 1, 1))

We will instantiate the FarmVibes.AI remote client and run the workflow:

In [ ]:
# Instantiate the client
client = get_default_vibe_client()

In this example, we will run a custom workflow that is defined in the `spaceeye_index.yaml` file. It runs the SpaceEye data ingestion workflow (`data_ingestion/spaceeye/spaceeye_interpolation`) and computes the NDVI (`data_processing/index/index`) on top of each daily cloud-free raster. Both the SpaceEye and the NDVI rasters are exposed as sinks. For more information on defining a custom workflow, refer to [the WORKFLOW documentation file](../../documentation/WORKFLOWS.md).

In [ ]:
RUN_NAME = "SpaceEye and NDVI Timelapse 2021"

with open("spaceeye_index.yaml") as f:
    wf_dict = yaml.safe_load(f)

wf_run = client.run(wf_dict, RUN_NAME, geometry=geometry, time_range=time_range)
wf_run.monitor()

Once finished, we can access the generated outputs through `wf_run.output`, using the sinks defined in `spaceeye_index.yaml` as keys:

In [ ]:
spaceeye_rasters = wf_run.output["spaceeye_raster"]
ndvi_rasters = wf_run.output["index"]

----------------------

### Visualizing the rasters

We will start by plotting a few of the SpaceEye images output by FarmVibes.AI: 

In [ ]:
# Sort assets by date
sorted_spaceeye_rasters = sorted(spaceeye_rasters, key=lambda x: x.time_range[0])

# Plot a few of them through the time_range
n_to_plot = 12
indices = np.linspace(0, len(sorted_spaceeye_rasters) - 1, n_to_plot, dtype=int)

fig, ax = plt.subplots(4, 3, figsize=(15, 12))
for ax_idx, idx in enumerate(indices):
    raster = sorted_spaceeye_rasters[idx]
    ar = spaceeye_to_img(read_raster(raster, geometry=geometry)[0])
    ax[ax_idx // 3, ax_idx % 3].set_title(f"{raster.time_range[0].strftime('%Y-%m-%d')}")
    ax[ax_idx // 3, ax_idx % 3].axis("off")
    ax[ax_idx // 3, ax_idx % 3].imshow(ar)
lw_plot()

Let's do the same for the NDVI:

In [ ]:
# Sort assets by date
sorted_ndvi_rasters = sorted(ndvi_rasters, key=lambda x: x.time_range[0])

# Plot a few of them through the time_range
fig, ax = plt.subplots(4, 3, figsize=(15, 12))

for ax_idx, idx in enumerate(indices):
    raster = sorted_ndvi_rasters[idx]
    ar = read_clip_index(raster, geometry)
    ax[ax_idx // 3, ax_idx % 3].set_title(f"{raster.time_range[0].strftime('%Y-%m-%d')}")
    ax[ax_idx // 3, ax_idx % 3].axis("off")
    ax[ax_idx // 3, ax_idx % 3].imshow(ar)

-------------------------------------

## Timelapse

To get a better sense of the progression of the region throughout the time range, let's build a timelapse from the generated images using `matplotlib.animation`. 

As we might have seen in the images above, some of them have missing data represented by black/white regions. We will filter those out before generating our timelapse.

In [ ]:
# Filtering out rasters with more than 10% of missing pixels
missing_data_thr = 0.1

missing_ratio = [get_missing_ratio(raster, geometry) for raster in sorted_spaceeye_rasters]
filtered_se_rasters = [
    se_raster
    for se_raster, r in zip(sorted_spaceeye_rasters, missing_ratio)
    if r < missing_data_thr
]
filtered_ndvi_rasters = [
    ndvi_raster
    for ndvi_raster, r in zip(sorted_ndvi_rasters, missing_ratio)
    if r < missing_data_thr
]

print(f"Filtered / Total rasters: {len(filtered_se_rasters)} / {len(sorted_spaceeye_rasters)}")

### SpaceEye timelapse

In [ ]:
# Set up the figure and axis
fig, ax = plt.subplots()

# Initialize the image plot
ar = spaceeye_to_img(read_raster(filtered_se_rasters[0], geometry)[0])
im = ax.imshow(ar)
ax.set_title(f"{filtered_se_rasters[0].time_range[0].strftime('%Y-%m-%d')}")
ax.axis("off")


# Function to update the plot for each frame
def update(i):
    raster = filtered_se_rasters[i]
    ar = spaceeye_to_img(read_raster(raster, geometry)[0])
    im.set_array(ar)
    ax.set_title(f"{raster.time_range[0].strftime('%Y-%m-%d')}")
    return (im,)


# Set up the animation using the update function
ani = animation.FuncAnimation(
    fig, update, frames=range(len(filtered_se_rasters)), blit=True, interval=33.33
)

# Save the animation as a GIF
ani.save("timelapse_spaceeye.gif", writer="imagemagick")

Once the generation has finished, we can inspect the file or visualize the timelapse GIF with the following code:

In [ ]:
# We opted not to provide the gifs in the notebook to avoid increasing the notebook size
display(Image(filename="./timelapse_spaceeye.gif"))

### NDVI timelapse

In [ ]:
# Set up the figure and axis
fig, ax = plt.subplots()

# Initialize the image plot
ar = read_clip_index(filtered_ndvi_rasters[0], geometry)

im = ax.imshow(ar)
ax.set_title(
    f"{filtered_ndvi_rasters[0].time_range[0].strftime('%Y-%m-%d')} --- mean NDVI {np.nanmean(ar):.3f}"
)
ax.axis("off")


# Function to update the plot for each frame
def update(i):
    raster = filtered_ndvi_rasters[i]
    ar = read_clip_index(raster, geometry)
    im.set_array(ar)
    ax.set_title(f"{raster.time_range[0].strftime('%Y-%m-%d')} --- mean NDVI {np.nanmean(ar):.3f}")
    return (im,)


# Set up the animation using the update function
ani = animation.FuncAnimation(
    fig, update, frames=range(len(filtered_ndvi_rasters)), blit=True, interval=33.33
)

# Save the animation as a GIF
ani.save("timelapse_ndvi.gif", writer="imagemagick")

In [ ]:
# We opted not to provide the gifs in the notebook to avoid increasing the notebook size
display(Image(filename="./timelapse_ndvi.gif"))